In [1]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage

from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraryies
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
#import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [2]:
#Import Data
nfl_data = nfl.import_schedules([2024])
original_df = nfl_data.groupby('week')
# Get data from games
weekly_nfl = nfl.import_weekly_data([2024])
nfl_this_season = nfl.import_weekly_data([2024])
#weekly_nfl.dtypes

Downcasting floats.
Downcasting floats.


In [3]:
# columns for receiving stats per game

columns = [ 'week','player_display_name','recent_team','opponent_team','receptions',
           'targets', 'receiving_yards','receiving_tds','receiving_epa','receiving_air_yards', 'receiving_yards_after_catch',
           'target_share', 'wopr', 'pacr'
           ]

In [4]:
#create Df
games_df = weekly_nfl[columns]
games_df.head()

,week,player_display_name,recent_team,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,1,Aaron Rodgers,NYJ,SF,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,1.018293
1,2,Aaron Rodgers,NYJ,TEN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,1.053892
2,3,Aaron Rodgers,NYJ,NE,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,1.243363
3,4,Aaron Rodgers,NYJ,DEN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,0.735294
4,5,Aaron Rodgers,NYJ,MIN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,0.595122


In [5]:
# Groupby opponent and week offense stats
team_by_week = games_df.groupby(['week', 'opponent_team','recent_team'])
team_by_week.head()
team_stats_per_game = team_by_week.sum()
team_stats_per_game.index.set_names(['week', 'team_defense','recent_team'], inplace=True)
team_stats_per_game = team_stats_per_game.reset_index()
team_stats_per_game.head()

,week,team_defense,recent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,1,ARI,BUF,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.397590
1,1,ATL,PIT,17,22,156.0,0,3.547840,135.0,54.0,1.0,2.2,1.218750
2,1,BAL,KC,20,27,291.0,1,14.509170,160.0,188.0,1.0,2.2,1.763636
3,1,BUF,ARI,21,31,162.0,1,8.129718,191.0,100.0,1.0,2.2,0.848168
4,1,CAR,NO,19,24,200.0,3,8.904821,208.0,96.0,1.0,2.2,1.000000


In [6]:
# add _def to columns
columns_to_suffix_def = [
    'receptions',
    'targets',
    'receiving_yards',
    'receiving_tds',
    'receiving_epa',
    'receiving_air_yards',
    'receiving_yards_after_catch',
    'target_share',
    'fantasy_points_ppr',
    'wopr',
    'pacr'
]

# Add "_def" to specific columns
team_stats_per_game = team_stats_per_game.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game.head()

,week,team_defense,recent_team,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,wopr_def,pacr_def
0,1,ARI,BUF,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.397590
1,1,ATL,PIT,17,22,156.0,0,3.547840,135.0,54.0,1.0,2.2,1.218750
2,1,BAL,KC,20,27,291.0,1,14.509170,160.0,188.0,1.0,2.2,1.763636
3,1,BUF,ARI,21,31,162.0,1,8.129718,191.0,100.0,1.0,2.2,0.848168
4,1,CAR,NO,19,24,200.0,3,8.904821,208.0,96.0,1.0,2.2,1.000000


In [7]:
# create receiver stats per player per week vs opp
# Create a DF for WR's and TE's and RB's vs the pass
wr_df_two = weekly_nfl[(weekly_nfl["position"] == "WR") | (weekly_nfl["position"] == "TE") | (weekly_nfl["position"] == "RB")]
wr_df = wr_df_two[columns]
#wr_df= wr_df[wr_df['fantasy_points_ppr'] != 0]
wr_df.head()


,week,player_display_name,recent_team,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
17,6,Marcedes Lewis,CHI,JAX,1,1,2.0,0,0.064605,1.0,1.0,0.038462,0.064488,NaN
99,1,Adam Thielen,CAR,NO,3,4,49.0,0,4.642717,64.0,18.0,0.137931,0.334169,NaN
100,2,Adam Thielen,CAR,LAC,2,3,20.0,0,-2.458903,18.0,2.0,0.115385,0.336713,NaN
101,3,Adam Thielen,CAR,LV,3,5,40.0,1,3.056416,44.0,5.0,0.135135,0.346628,NaN
102,12,Adam Thielen,CAR,KC,3,4,57.0,0,4.730047,34.0,35.0,0.121212,0.263605,NaN


In [8]:
#groupby team and opponent and player
wr_v_d = wr_df.groupby(['opponent_team','recent_team', 'player_display_name','week']).mean()
wr_v_d.head()

receptions  targets  \
opponent_team recent_team player_display_name week                        
ARI           BUF         Curtis Samuel       1            2.0      2.0   
                          Dalton Kincaid      1            1.0      2.0   
                          Dawson Knox         1            1.0      2.0   
                          James Cook          1            3.0      3.0   
                          Keon Coleman        1            4.0      5.0   

                                                    receiving_yards  \
opponent_team recent_team player_display_name week                    
ARI           BUF         Curtis Samuel       1                15.0   
                          Dalton Kincaid      1                11.0   
                          Dawson Knox         1                23.0   
                          James Cook          1                32.0   
                          Keon Coleman        1                51.0   

                                                    receiving_tds  \
opponent_team recent_team player_display_name week                  
ARI           BUF         Curtis Samuel       1               0.0   
                          Dalton Kincaid      1               0.0   
                          Dawson Knox         1               0.0   
                          James Cook          1               0.0   
                          Keon Coleman        1               0.0   

                                                    receiving_epa  \
opponent_team recent_team player_display_name week                  
ARI           BUF         Curtis Samuel       1          0.546278   
                          Dalton Kincaid      1          0.286692   
                          Dawson Knox         1          1.050424   
                          James Cook          1          2.449019   
                          Keon Coleman        1          3.296488   

                                                    receiving_air_yards  \
opponent_team recent_team player_display_name week                        
ARI           BUF         Curtis Samuel       1                     3.0   
                          Dalton Kincaid      1                     1.0   
                          Dawson Knox         1                    19.0   
                          James Cook          1                     1.0   
                          Keon Coleman        1                    46.0   

                                                    receiving_yards_after_catch  \
opponent_team recent_team player_display_name week                                
ARI           BUF         Curtis Samuel       1                            12.0   
                          Dalton Kincaid      1                             6.0   
                          Dawson Knox         1                            17.0   
                          James Cook          1                            31.0   
                          Keon Coleman        1                            11.0   

                                                    target_share      wopr  \
opponent_team recent_team player_display_name week                           
ARI           BUF         Curtis Samuel       1         0.086957  0.143085   
                          Dalton Kincaid      1         0.086957  0.134652   
                          Dawson Knox         1         0.086957  0.210555   
                          James Cook          1         0.130435  0.199869   
                          Keon Coleman        1         0.217391  0.520063   

                                                    pacr  
opponent_team recent_team player_display_name week        
ARI           BUF         Curtis Samuel       1      NaN  
                          Dalton Kincaid      1      NaN  
                          Dawson Knox         1      NaN  
                          James Cook          1      NaN  
                          Keon Coleman        1    

In [9]:
#reset index
wr_v_d_df = wr_v_d.reset_index()
wr_v_d_df.head()

,opponent_team,recent_team,player_display_name,week,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,ARI,BUF,Curtis Samuel,1,2.0,2.0,15.0,0.0,0.546278,3.0,12.0,0.086957,0.143085,NaN
1,ARI,BUF,Dalton Kincaid,1,1.0,2.0,11.0,0.0,0.286692,1.0,6.0,0.086957,0.134652,NaN
2,ARI,BUF,Dawson Knox,1,1.0,2.0,23.0,0.0,1.050424,19.0,17.0,0.086957,0.210555,NaN
3,ARI,BUF,James Cook,1,3.0,3.0,32.0,0.0,2.449019,1.0,31.0,0.130435,0.199869,NaN
4,ARI,BUF,Keon Coleman,1,4.0,5.0,51.0,0.0,3.296488,46.0,11.0,0.217391,0.520063,NaN


In [10]:
#change opponent team name to team_defense
wr_v_d_df = wr_v_d_df.rename(columns={'opponent_team': 'team_defense'})


In [11]:
# merge players stats per game vs the defense stats that game

test_data_df = wr_v_d_df.merge(team_stats_per_game, on=['week', 'team_defense', 'recent_team'])
test_data_df = test_data_df.fillna(0)


In [12]:
test_data_df.head(10)

,team_defense,recent_team,player_display_name,week,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,...,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,wopr_def,pacr_def
0,ARI,BUF,Curtis Samuel,1,2.0,2.0,15.0,0.0,0.546278,3.0,...,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.39759
1,ARI,BUF,Dalton Kincaid,1,1.0,2.0,11.0,0.0,0.286692,1.0,...,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.39759
2,ARI,BUF,Dawson Knox,1,1.0,2.0,23.0,0.0,1.050424,19.0,...,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.39759
3,ARI,BUF,James Cook,1,3.0,3.0,32.0,0.0,2.449019,1.0,...,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.39759
4,ARI,BUF,Keon Coleman,1,4.0,5.0,51.0,0.0,3.296488,46.0,...,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.39759
5,ARI,BUF,Khalil Shakir,1,3.0,3.0,42.0,1.0,3.878293,12.0,...,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.39759
6,ARI,BUF,Mack Hollins,1,2.0,2.0,25.0,1.0,3.893081,24.0,...,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.39759
7,ARI,BUF,Marquez Valdes-Scantling,1,1.0,2.0,19.0,0.0,-0.206561,55.0,...,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.39759
8,ARI,BUF,Ray Davis,1,1.0,1.0,14.0,0.0,1.892505,2.0,...,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.39759
9,ARI,BUF,Ty Johnson,1,0.0,1.0,0.0,0.0,-1.403000,3.0,...,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.39759


In [13]:
# create test data 

x_cols_train = ['receiving_epa',
                 'target_share',
                'receiving_epa_def', 'targets_def'
                
    
]


y_cols_train = ['receiving_yards','receiving_tds','receptions']

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# model
# Split data into features (X) and target variable (y)
X = test_data_df[x_cols_train]
y = test_data_df[y_cols_train]

# Encode categorical variables using one-hot encoding if needed

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")

Mean Absolute Error (MAE): 3.354235676379235
Mean Squared Error (MSE): 61.442738767961465
Root Mean Squared Error (RMSE): 4.942560911502059
R-squared (R2): 0.6601418361538575


In [15]:
#create Df
games_df_test = nfl_this_season[columns]
games_df_test.head()

,week,player_display_name,recent_team,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,1,Aaron Rodgers,NYJ,SF,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,1.018293
1,2,Aaron Rodgers,NYJ,TEN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,1.053892
2,3,Aaron Rodgers,NYJ,NE,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,1.243363
3,4,Aaron Rodgers,NYJ,DEN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,0.735294
4,5,Aaron Rodgers,NYJ,MIN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,0.595122


In [16]:
# Groupby opponent and week offense stats
team_by_week_test = games_df_test.groupby(['week', 'opponent_team','recent_team'])
#team_by_week.head()
team_stats_per_game_test = team_by_week_test.sum()
team_stats_per_game_test.index.set_names(['week', 'team_defense','recent_team'], inplace=True)
team_stats_per_game_test = team_stats_per_game_test.reset_index()
team_stats_per_game_test.head()

,week,team_defense,recent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,1,ARI,BUF,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.397590
1,1,ATL,PIT,17,22,156.0,0,3.547840,135.0,54.0,1.0,2.2,1.218750
2,1,BAL,KC,20,27,291.0,1,14.509170,160.0,188.0,1.0,2.2,1.763636
3,1,BUF,ARI,21,31,162.0,1,8.129718,191.0,100.0,1.0,2.2,0.848168
4,1,CAR,NO,19,24,200.0,3,8.904821,208.0,96.0,1.0,2.2,1.000000


In [17]:
# Add "_def" to specific columns
team_stats_per_game_test = team_stats_per_game_test.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game_test.head()

,week,team_defense,recent_team,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,wopr_def,pacr_def
0,1,ARI,BUF,18,23,232.0,2,15.683219,166.0,125.0,1.0,2.2,1.397590
1,1,ATL,PIT,17,22,156.0,0,3.547840,135.0,54.0,1.0,2.2,1.218750
2,1,BAL,KC,20,27,291.0,1,14.509170,160.0,188.0,1.0,2.2,1.763636
3,1,BUF,ARI,21,31,162.0,1,8.129718,191.0,100.0,1.0,2.2,0.848168
4,1,CAR,NO,19,24,200.0,3,8.904821,208.0,96.0,1.0,2.2,1.000000


In [18]:
# Create a DF for WR's and TE's and RB's vs the pass
wr_df_two_test = nfl_this_season[(nfl_this_season["position"] == "WR") | (nfl_this_season["position"] == "TE") | (nfl_this_season["position"] == "RB")]
wr_df_test = wr_df_two_test[columns]
#wr_df= wr_df[wr_df['fantasy_points_ppr'] != 0]
wr_df_test.head()

,week,player_display_name,recent_team,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
17,6,Marcedes Lewis,CHI,JAX,1,1,2.0,0,0.064605,1.0,1.0,0.038462,0.064488,NaN
99,1,Adam Thielen,CAR,NO,3,4,49.0,0,4.642717,64.0,18.0,0.137931,0.334169,NaN
100,2,Adam Thielen,CAR,LAC,2,3,20.0,0,-2.458903,18.0,2.0,0.115385,0.336713,NaN
101,3,Adam Thielen,CAR,LV,3,5,40.0,1,3.056416,44.0,5.0,0.135135,0.346628,NaN
102,12,Adam Thielen,CAR,KC,3,4,57.0,0,4.730047,34.0,35.0,0.121212,0.263605,NaN


In [19]:
#groupby team and opponent and player
wr_v_d_test = wr_df_test.groupby(['opponent_team','recent_team', 'player_display_name','week']).mean()
#reset index
wr_v_d_df_test = wr_v_d_test.reset_index()
wr_v_d_df_test = wr_v_d_df_test.rename(columns={'opponent_team': 'team_defense'})
wr_v_d_df_test.head()


,team_defense,recent_team,player_display_name,week,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,ARI,BUF,Curtis Samuel,1,2.0,2.0,15.0,0.0,0.546278,3.0,12.0,0.086957,0.143085,NaN
1,ARI,BUF,Dalton Kincaid,1,1.0,2.0,11.0,0.0,0.286692,1.0,6.0,0.086957,0.134652,NaN
2,ARI,BUF,Dawson Knox,1,1.0,2.0,23.0,0.0,1.050424,19.0,17.0,0.086957,0.210555,NaN
3,ARI,BUF,James Cook,1,3.0,3.0,32.0,0.0,2.449019,1.0,31.0,0.130435,0.199869,NaN
4,ARI,BUF,Keon Coleman,1,4.0,5.0,51.0,0.0,3.296488,46.0,11.0,0.217391,0.520063,NaN


In [20]:
# merge players stats per game vs the defense stats that game

test_data_df_test = wr_v_d_df_test.merge(team_stats_per_game_test, on=['week', 'team_defense', 'recent_team'])
test_data_df_test = test_data_df_test.fillna(0)

In [21]:
# Create the averages for receivers
## First Create a mean of Each Players stats 

player_mean = wr_df_test.groupby('player_display_name').mean()
#remove week
player_mean.drop("week", axis=1, inplace=True)


player_mean.head()


,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
player_display_name,,,,,,,,,,
A.J. Brown,4.857143,7.142857,77.785713,0.500000,4.282692,87.000000,25.214285,0.326275,0.839650,NaN
AJ Barner,2.142857,2.714286,17.500000,0.285714,0.791903,9.714286,11.714286,0.085408,0.165533,NaN
Aaron Jones,3.000000,3.777778,23.333334,0.111111,0.052095,1.666667,23.833334,0.123298,0.188549,NaN
Aaron Shampklin,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN
Adam Thielen,4.800000,6.200000,61.500000,0.500000,4.042908,69.199997,15.200000,0.204031,0.489588,NaN


In [22]:
# Add recent_team to player_mean
team_cols = ['player_display_name', 'recent_team']
team_player = test_data_df_test[team_cols]
# merge to player_mean
player_mean_df = player_mean.merge(team_player, on='player_display_name')
player_mean_df = player_mean_df.drop_duplicates()
player_mean_df = player_mean_df.fillna(0)
player_mean_df.head()

,player_display_name,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr,recent_team
0,A.J. Brown,4.857143,7.142857,77.785713,0.500000,4.282692,87.000000,25.214285,0.326275,0.839650,0.0,PHI
14,AJ Barner,2.142857,2.714286,17.500000,0.285714,0.791903,9.714286,11.714286,0.085408,0.165533,0.0,SEA
28,Aaron Jones,3.000000,3.777778,23.333334,0.111111,0.052095,1.666667,23.833334,0.123298,0.188549,0.0,MIN
46,Aaron Shampklin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,PIT
49,Adam Thielen,4.800000,6.200000,61.500000,0.500000,4.042908,69.199997,15.200000,0.204031,0.489588,0.0,CAR


In [23]:
# now get teams average for these stats
def_mean = wr_df_test.groupby(['week', 'opponent_team']).sum()

#get mean of team vs receiving stats
total_sum_def = def_mean.reset_index()
team_sum_stats = total_sum_def.groupby('opponent_team').mean()
team_sum_stats.drop("week", axis=1, inplace=True)

In [24]:
# Add "_def" to specific columns
team_sum_stats = team_sum_stats.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_sum_stats = team_sum_stats.reset_index()
team_sum_stats.head()

,opponent_team,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,wopr_def,pacr_def
0,ARI,21.411765,29.705882,224.941177,1.058824,7.646708,209.823532,110.529411,0.986699,2.177697,0.000000
1,ATL,23.529412,32.176471,235.294113,2.000000,7.723002,227.352936,114.411766,0.992034,2.187457,0.128676
2,BAL,22.833333,34.111111,260.722229,1.555556,6.865835,282.388885,113.055557,0.991814,2.186406,0.055556
3,BUF,22.666667,31.611111,235.833328,1.555556,7.467795,224.944443,115.111115,0.997179,2.195313,0.000000
4,CAR,21.117647,29.705882,236.176468,2.058824,9.097728,227.176468,110.705879,0.996351,2.192786,0.096886


In [25]:
# create DF for upcoming week
# create stats to show upcpoming games
grouped = nfl_data.groupby('week')

# Select the group corresponding to week 6
week_6_data = grouped.get_group(20)
get_teams =['home_team','away_team']
schedule_week_df = week_6_data[get_teams]
schedule_week_df.reset_index(drop=True, inplace=True)
schedule_week_df.head()

,home_team,away_team
0,KC,HOU
1,DET,WAS
2,PHI,LA
3,BUF,BAL


In [26]:
# rename cols 
schedule_week_df = schedule_week_df.rename(columns={'home_team': 'recent_team', 'away_team': 'opponent_team'})
schedule_week_df.head()

,recent_team,opponent_team
0,KC,HOU
1,DET,WAS
2,PHI,LA
3,BUF,BAL


In [27]:
# Create a reversed DataFrame
reversed_df = pd.DataFrame({'recent_team': schedule_week_df['opponent_team'], 'opponent_team': schedule_week_df['recent_team'],})

# Concatenate the original and reversed DataFrames
final_week_df = pd.concat([schedule_week_df, reversed_df], ignore_index=True)

final_week_df.head(23)

,recent_team,opponent_team
0,KC,HOU
1,DET,WAS
2,PHI,LA
3,BUF,BAL
4,HOU,KC
5,WAS,DET
6,LA,PHI
7,BAL,BUF


In [28]:
# merge team_sum_stats with final_week_df

testing_def = final_week_df.merge(team_sum_stats, on='opponent_team')
testing_def.head()

,recent_team,opponent_team,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,wopr_def,pacr_def
0,KC,HOU,18.833333,31.333333,222.000000,1.777778,2.815909,290.944458,117.000000,0.998971,2.197991,0.000000
1,DET,WAS,17.777778,27.166667,203.722229,1.500000,7.283944,234.944443,90.555557,0.994584,2.190211,0.058333
2,PHI,LA,20.888889,30.833333,238.722229,1.611111,7.763112,256.388885,117.000000,0.995258,2.191130,0.000000
3,BUF,BAL,22.833333,34.111111,260.722229,1.555556,6.865835,282.388885,113.055557,0.991814,2.186406,0.055556
4,HOU,KC,22.000000,31.705882,233.117645,1.411765,7.133181,245.411758,118.411766,0.990539,2.179256,0.000000


In [29]:
# merge player_mean_df with testing_def

final_test_df = testing_def.merge(player_mean_df, on='recent_team')
# Replace NaN values with 0
final_test_df = final_test_df.fillna(0)


In [30]:
final_test_df.head(16)

,recent_team,opponent_team,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,...,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,KC,HOU,18.833333,31.333333,222.0,1.777778,2.815909,290.944458,117.0,0.998971,...,0.000000,1.000000,0.000000,0.000000,-0.678180,0.000000,0.000000,0.027778,0.041667,0.0
1,KC,HOU,18.833333,31.333333,222.0,1.777778,2.815909,290.944458,117.0,0.998971,...,0.583333,0.916667,2.166667,0.000000,-0.483748,-1.416667,3.583333,0.066122,0.086804,0.0
2,KC,HOU,18.833333,31.333333,222.0,1.777778,2.815909,290.944458,117.0,0.998971,...,3.500000,5.000000,38.125000,0.312500,1.200577,56.000000,6.812500,0.144948,0.383617,0.0
3,KC,HOU,18.833333,31.333333,222.0,1.777778,2.815909,290.944458,117.0,0.998971,...,1.714286,2.285714,11.285714,0.000000,-0.308422,-3.142857,17.000000,0.089489,0.107027,0.0
4,KC,HOU,18.833333,31.333333,222.0,1.777778,2.815909,290.944458,117.0,0.998971,...,1.000000,1.000000,7.000000,0.000000,-0.145088,4.000000,3.000000,0.043478,0.097036,0.0
5,KC,HOU,18.833333,31.333333,222.0,1.777778,2.815909,290.944458,117.0,0.998971,...,0.333333,1.000000,1.666667,0.000000,-1.889073,1.333333,0.666667,0.032661,0.053768,0.0
6,KC,HOU,18.833333,31.333333,222.0,1.777778,2.815909,290.944458,117.0,0.998971,...,1.636364,2.363636,21.000000,0.181818,0.116790,13.000000,14.090909,0.072261,0.158834,0.0
7,KC,HOU,18.833333,31.333333,222.0,1.777778,2.815909,290.944458,117.0,0.998971,...,1.466667,2.133333,19.266666,0.133333,1.272464,32.400002,4.533333,0.068279,0.206823,0.0
8,KC,HOU,18.833333,31.333333,222.0,1.777778,2.815909,290.944458,117.0,0.998971,...,0.000000,1.000000,0.000000,0.000000,-1.216528,10.000000,0.000000,0.025000,0.075751,0.0
9,KC,HOU,18.833333,31.333333,222.0,1.777778,2.815909,290.944458,117.0,0.998971,...,1.769231,2.461538,13.538462,0.000000,0.274292,-4.461538,16.615385,0.079320,0.103596,0.0


In [31]:
# Now, use the trained model to predict rb_yards for df_test
X_test = final_test_df[x_cols_train]
y_pred = model.predict(X_test)

# The predictions are stored in y_pred
print("Predicted Scores:")
#print(y_pred)

Predicted Scores:


In [32]:
# add predicted scores into df: 
predict_cols = ['recent_team','opponent_team','player_display_name']
df_predictions = final_test_df[predict_cols]

In [33]:
scores_df = pd.DataFrame(y_pred, columns=['receiving_yards','receiving_tds','receptions'])
df_predictions = pd.concat([df_predictions, scores_df], axis=1)

In [34]:
df_predictions.head(50)
df_predictions.to_excel('wr_model_four_ex.xlsx', index=True)

In [ ]:
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def display_dataframe():
    return df_predictions.to_html()

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jan/2025 09:57:06] "GET / HTTP/1.1" 200 -
